# Assignment  Practice Text classification with Naive Bayes 

## Notebook made by   (If not filled in correctly: 0 pts for assignment)

__Name(s)__: Luca Verhees & Mitchell Kuin

__Student id(s)__ : 10837833 & 10764631

__Email(s)__: lucaverhees@gmail.com & mitchell_kuin@hotmail.com

### Pledge (taken from [Coursera's Honor Code](https://www.coursera.org/about/terms/honorcode) )



Put here a selfie with your photo where you hold a signed paper with the following text: (if this is team work, put two selfies here). The link must be to some place on the web, not to a local file. **Assignments without the selfies will not be graded and receive 0 points.**

> My answers to homework, quizzes and exams will be my own work (except for assignments that explicitly permit collaboration).

>I will not make solutions to homework, quizzes or exams available to anyone else. This includes both solutions written by me, as well as any official solutions provided by the course staff.

>I will not engage in any other activities that will dishonestly improve my results or dishonestly improve/hurt the results of others.

<img src='http://www.lucaverhees.nl/pledge.jpg' width="400px"/>
<img src='http://i.imgur.com/asqxf82.jpg' width="400px">

# Practice Text classification with Naive Bayes  
        
        
        
<h3>Abstract</h3>
<p>We will do text classification on a collection of Dutch parliamentary questions.
    The website <a href="https://zoek.officielebekendmakingen.nl/zoeken/parlementaire_documenten">officielebekendmakingen.nl</a>lets you search in "kamervragen".
    You can donwload
    <a href='http://data.politicalmashup.nl/kamervragen/PoliDocs_Kamervragen.zip'>this zipfile with Kamervragen in XML</a>
    to see some of the  data in XML format. 
    It also contains style sheets to show the XML well in a browser.  
    The <a href='http://maartenmarx.nl/teaching/zoekmachines/LectureNotes/MySQL/'>MYSQL directory</a> contains an <a href='http://maartenmarx.nl/teaching/zoekmachines/LectureNotes/MySQL/KVR14807.xml'>example   Kamervraag XML file</a> and a file `kvr.csv.gz` with all those kamervragen in a handy csv format. Note that in your browser you see the result of applying stylesheets. So choose View Source or open it in an editor.</p>

<h3>First exploration</h3>

See below.

<h2>Exercise</h2>

<p>We will use the fields in elements of the form <tt> &lt;item attribuut="Afkomstig_van"></tt> as our classes. 
    These are the ministeries to whom the question is addressed.
    An example is 
    <pre>
        &lt;item attribuut="Afkomstig_van">Landbouw, Natuurbeheer en Visserij (LNV)&lt;/item>
    </pre>
    Note that these labels are <strong>not normalized</strong>, see e.g. the counts below:
    <pre>
Justitie (JUS)                                                   3219
Volksgezondheid, Welzijn en Sport (VWS)                          2630
Buitenlandse Zaken (BUZA)                                        1796
Verkeer en Waterstaat (VW)                                       1441
Justitie                                                         1333
Sociale Zaken en Werkgelegenheid (SZW)                           1231
Onderwijs, Cultuur en Wetenschappen (OCW)                        1187
Volkshuisvesting, Ruimtelijke Ordening en Milieubeheer (VROM)     984
FinanciÃ«n (FIN)                                                   960
Volksgezondheid, Welzijn en Sport                                 951
Economische Zaken (EZ)                                            946
Buitenlandse Zaken                                                753
Binnenlandse Zaken en Koninkrijksrelaties (BZK)                   725
Verkeer en Waterstaat                                             724
Defensie (DEF)                                                    646
Sociale Zaken en Werkgelegenheid                                  607
Landbouw, Natuurbeheer en Visserij (LNV)                          586
Volkshuisvesting, Ruimtelijke Ordening en Milieubeheer            554
Onderwijs, Cultuur en Wetenschappen                               532
Vreemdelingenzaken en Integratie (VI)                             466
    </pre>
</p>

  <ol>
      <li>Normalize the values for "ministerie" and choose 10 ministeries to work with. </li>
      <li>Implement the two algorithms in Fig MRS.13.2, using your earlier code for creating term and document frequencies.
      It might be easier to use the representation and formula given in MRS section 13.4.1.</li>
      <li>On this collection, train NB text classifiers for 10 different classes with enough and interesting data.</li>
      <li>Compute for each term and each of your 10 classes its utility for that class using mutual information.</li>
      <li>For each class, show the top 10 words as in Figure 13.7 in MRS.</li>
      <li>Evaluate your classifiers using Precision, Recall and F1. (
           <br/>
          Give a table in which you show these values for using the top 10, top 100 terms and all terms, for all of your 10 classes.
          Thus do feature selection per class, and use for each class the top n best features for that class. 
          <br/>
      Also show the microaverage(s) for all 10 classes together.
      <br/>
      If you like you can also present this in a figure like MRS.13.8. 
      Then compute the F1 measure for the same number of terms as in that figure.</li>
      <li>Reflect and report briefly about your choices in this process and about the obtained results. </li>
  </ol>

<h3>Training/Testing</h3>
<p>It is important that you do not test your classifier using documents that have also been used in training.
    So split up your collection in a training set and a test set. A 80%-20% split is reasonable.

<br/>
    If you have too little data you can use 5 or <a href="http://en.wikipedia.org/wiki/Cross-validation_(statistics)#k-fold_cross-validation">10-fold cross validation</a>.</p>

<h2>Form of presentation</h2>
<ul>
    <li>Make slides or wikipages and have your system running (this could just be an IPython notebook with a classify function that accepts any string and classifies it.) ~~and be able to accept documents from the web.~~ </li>
    <li>Create one or two slides or wikipages for each of the sub exercises listed above.
</li>
<li>Make it clear in the heading of the slides which sub exercises you talk about.</li>
    <li>Show running code with one or two  good examples (a TP of course, but also a FP and an error-analysis is nice to show). </li>

</ul>

<h2>Form of handing in your final product</h2>
<ul>
    <li>An IPython notebook would be perfect, with clear indications which part of the code answers which subquestion.</li>
    <li>A clear git repo, with good comments and a clear separation and indication what code does what is also fine.</li>
    <li> You are free to program in whatever language you prefer.</li>
</ul>


## Preparing the data

In [35]:
import pandas as pd
import re
import nltk
from collections import defaultdict
import numpy as np
import math
from sklearn.model_selection import train_test_split

In [40]:
# Change to KVR1000.csv.gz if this becomes too slow for you
kvrdf = pd.read_csv('http://maartenmarx.nl/teaching/zoekmachines/LectureNotes/MySQL/KVR1000.csv.gz', 
                   compression='gzip', sep='\t', encoding='utf-8',
                   index_col=0, names=['jaar', 'partij','titel','vraag','antwoord','ministerie']) 
print kvrdf.shape
kvrdf.head()

(1000, 6)


,jaar,partij,titel,vraag,antwoord,ministerie
KVR1000.xml,1994,PvdA,De vragen betreffen de betrouwbaarheid van de...,Hebt u kennisgenomen van het televisieprogram...,Ja. Het bedoelde geluidmeetpunt is eigendom v...,Verkeer en Waterstaat
KVR10000.xml,1999,PvdA,Vragen naar aanleiding van berichten (uitzend...,Kent u de berichten over de situatie in de Me...,,Justitie
KVR10001.xml,1999,SP,"Vragen naar aanleiding van de berichten ""Nede...",Kent u de berichten «Nederland steunt de Soeh...,,Financien
KVR10002.xml,1999,PvdA,Vragen over de gebrekkige opvang van verpleeg...,Kent u het bericht over onderzoek van Nu91 me...,Ja. Het onderzoek van NU’91 wijst uit dat het...,"Volksgezondheid, Welzijn en Sport"
KVR10003.xml,1999,PvdA,Vragen over onbetrouwbaarheid van filemeldingen.,Hebt u kennisgenomen van de berichten over de...,Ja. Nee. Door de waarnemers van het Algemeen ...,Verkeer en Waterstaat


## Exercise 1
> Normalize the values for "ministerie" and choose 10 ministeries to work with.

In [41]:
kvrdf.ministerie = kvrdf.ministerie.str.replace('Justitie( \(JUS\))?', 'Justitie (JUS)')
kvrdf.ministerie = kvrdf.ministerie.str.replace('Volksgezondheid, Welzijn en Sport( \(VWS\))?', 'Volksgezondheid, Welzijn en Sport (VWS)')
kvrdf.ministerie = kvrdf.ministerie.str.replace('Buitenlandse Zaken( \(BUZA\))?', 'Buitenlandse Zaken (BUZA)')
kvrdf.ministerie = kvrdf.ministerie.str.replace('Verkeer en Waterstaat( \(VW\))?', 'Verkeer en Waterstaat (VW)')
kvrdf.ministerie = kvrdf.ministerie.str.replace('Sociale Zaken en Werkgelegenheid( \(SZW\))?', 'Sociale Zaken en Werkgelegenheid (SZW)')
kvrdf.ministerie = kvrdf.ministerie.str.replace('Onderwijs, Cultuur en Wetenschappen( \(OCW\))?', 'Onderwijs, Cultuur en Wetenschappen (OCW)')
kvrdf.ministerie = kvrdf.ministerie.str.replace('Volkshuisvesting, Ruimtelijke Ordening en Milieubeheer( \(VROM\))?', 'Volkshuisvesting, Ruimtelijke Ordening en Milieubeheer (VROM)')
kvrdf.ministerie = kvrdf.ministerie.str.replace('Financi.*n( \(FIN\))?', 'Financien (FIN)')
kvrdf.ministerie = kvrdf.ministerie.str.replace('Economische Zaken( \(EZ\))?', 'Economische Zaken (EZ)')
kvrdf.ministerie = kvrdf.ministerie.str.replace('Landbouw, Natuurbeheer en Visserij( \(LNV\))?', 'Landbouw, Natuurbeheer en Visserij (LNV)')

In [42]:
# Find top 10 ministeries
ministeries = kvrdf.ministerie.value_counts().head(10).keys()

# Create a new dataframe with only the top 10 ministeries
kvrdf2 = kvrdf[kvrdf['ministerie'].isin(ministeries)]

# Fix the random spaces occuring in the dataset
kvrdf2.index = [i.replace(" ", "") for i in kvrdf2.index]
kvrdf2.ministerie = [i.strip() for i in kvrdf2.ministerie]
ministeries = [m.strip() for m in ministeries]

print ministeries
print kvrdf2.shape
kvrdf2.head()

[u'Justitie (JUS)', u'Volksgezondheid, Welzijn en Sport (VWS)', u'Buitenlandse Zaken (BUZA)', u'Verkeer en Waterstaat (VW)', u'Sociale Zaken en Werkgelegenheid (SZW)', u'Onderwijs, Cultuur en Wetenschappen (OCW)', u'Volkshuisvesting, Ruimtelijke Ordening en Milieubeheer (VROM)', u'Landbouw, Natuurbeheer en Visserij (LNV)', u'Economische Zaken (EZ)', u'Financien (FIN)']
(838, 6)


,jaar,partij,titel,vraag,antwoord,ministerie
KVR1000.xml,1994,PvdA,De vragen betreffen de betrouwbaarheid van de...,Hebt u kennisgenomen van het televisieprogram...,Ja. Het bedoelde geluidmeetpunt is eigendom v...,Verkeer en Waterstaat (VW)
KVR10000.xml,1999,PvdA,Vragen naar aanleiding van berichten (uitzend...,Kent u de berichten over de situatie in de Me...,,Justitie (JUS)
KVR10001.xml,1999,SP,"Vragen naar aanleiding van de berichten ""Nede...",Kent u de berichten «Nederland steunt de Soeh...,,Financien (FIN)
KVR10002.xml,1999,PvdA,Vragen over de gebrekkige opvang van verpleeg...,Kent u het bericht over onderzoek van Nu91 me...,Ja. Het onderzoek van NU’91 wijst uit dat het...,"Volksgezondheid, Welzijn en Sport (VWS)"
KVR10003.xml,1999,PvdA,Vragen over onbetrouwbaarheid van filemeldingen.,Hebt u kennisgenomen van de berichten over de...,Ja. Nee. Door de waarnemers van het Algemeen ...,Verkeer en Waterstaat (VW)


## Exercise 2
> Implement the two algorithms in Fig MRS.13.2, using your earlier code for creating term and document frequencies. It might be easier to use the representation and formula given in MRS section 13.4.1.

In [43]:
def TrainMultinomialNB(C, D):
    class_vocabulary = defaultdict(list) # Initialize dictionary to store all text per class
    V = [] # Initialize list for vocabulary
    prior = {} # initialize list for prior scores
    condprob = defaultdict(lambda: defaultdict(dict))

    for row in D.itertuples():
        class_vocabulary[row[6]] += row[7] # add tokenized text for every doc to corresponding class
        V += row[7] # add tokenized text to vocabulary

    V = set(V) # vocabulary of corpus
    N = D.shape[0] # number of documents
    
    for c in C:
        Nc = len(D.loc[D.ministerie == c]) # number of documents in class c
        prior[c] = Nc / float(N) # save prior score to dict
    
        textc = class_vocabulary[c] # all text of class c
    
        for t in V: 
            Tct = textc.count(t) # term frequency of term t in collection c
            B = len(V) # size of the vocabulary
            condprob[t][c] = (Tct + 1.0)/(len(textc)+B) # save conditional probability score to dict
            
    return V, prior, condprob

In [72]:
def ApplyMultinomialNB(C, V, prior, condprob, d):
    scores = {} # initialize list for scores
    for c in C:
        score = math.log10(prior[c]) # start with Pc score
        for t in tokenize(d):
            if t in V:
                score += math.log10(condprob[t][c]) # multiply Pc score with every Ptk score
            else:
                score += math.log10((1.0 / len(V))) # this is for when a term doesn't occur in the vocabulary (so there is no Ptk score) 
        scores[c] = score
    v = list(scores.values())
    k = list(scores.keys())
    return k[v.index(max(v))]

## Exercise 3
> On this collection, train NB text classifiers for 10 different classes with enough and interesting data.

In [45]:
# Function to tokennize text
def tokenize(text):
    return [re.sub(r'[^\w\s]','',w) for w in nltk.word_tokenize(text.lower()) if re.sub(r'[^\w\s]','',w) != '']
   
# Create a new column with contents from 'titel', 'vraag' and 'antwoord' tokenized 
kvrdf2['tokenized'] = kvrdf2.apply(lambda row: tokenize(row.titel+row.vraag+row.antwoord), axis=1)
kvrdf2.head()

/Users/lucaverhees/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,jaar,partij,titel,vraag,antwoord,ministerie,tokenized
KVR1000.xml,1994,PvdA,De vragen betreffen de betrouwbaarheid van de...,Hebt u kennisgenomen van het televisieprogram...,Ja. Het bedoelde geluidmeetpunt is eigendom v...,Verkeer en Waterstaat (VW),"[de, vragen, betreffen, de, betrouwbaarheid, v..."
KVR10000.xml,1999,PvdA,Vragen naar aanleiding van berichten (uitzend...,Kent u de berichten over de situatie in de Me...,,Justitie (JUS),"[vragen, naar, aanleiding, van, berichten, uit..."
KVR10001.xml,1999,SP,"Vragen naar aanleiding van de berichten ""Nede...",Kent u de berichten «Nederland steunt de Soeh...,,Financien (FIN),"[vragen, naar, aanleiding, van, de, berichten,..."
KVR10002.xml,1999,PvdA,Vragen over de gebrekkige opvang van verpleeg...,Kent u het bericht over onderzoek van Nu91 me...,Ja. Het onderzoek van NU’91 wijst uit dat het...,"Volksgezondheid, Welzijn en Sport (VWS)","[vragen, over, de, gebrekkige, opvang, van, ve..."
KVR10003.xml,1999,PvdA,Vragen over onbetrouwbaarheid van filemeldingen.,Hebt u kennisgenomen van de berichten over de...,Ja. Nee. Door de waarnemers van het Algemeen ...,Verkeer en Waterstaat (VW),"[vragen, over, onbetrouwbaarheid, van, filemel..."


In [46]:
# splitting dataframe to train (80%) and test data (20%)
train, test = train_test_split(kvrdf2, test_size=0.2)

In [47]:
# the classes for the algorithm
classes = ministeries

In [48]:
# Train NB text classifiers for 10 different classes
%time trainResult = TrainMultinomialNB(classes, train)

CPU times: user 4min 29s, sys: 1.82 s, total: 4min 31s
Wall time: 5min 39s


In [75]:
# Test trained classifiers
test_doc = test.titel[0] + test.vraag[0] + test.antwoord[0]
print "Class should be:", test.ministerie[0]

Class should be: Buitenlandse Zaken (BUZA)


In [76]:
%time result = ApplyMultinomialNB(classes, trainResult[0], trainResult[1], trainResult[2], test_doc)
print "Classified class is:", result

CPU times: user 66.9 ms, sys: 10.3 ms, total: 77.2 ms
Wall time: 69.7 ms
Classified class is: Buitenlandse Zaken (BUZA)


## Exercise 4 
> Compute for each term and each of your 10 classes its utility for that class using mutual information. 

In [104]:
def computeUtilityMI(V,C):
    
    scores = defaultdict(lambda: defaultdict(dict)) # initialize dict for utility scores
    N = float(train.shape[0]) # the size of the vocabulary
    
    for t in V:
        for c in C:
            N11 = 0 # contains term and is in class
            N00 = 0 # doesn't contain term and is not in class
            N10 = 0 # contains term and is not in class
            N01 = 0 # doesn't contain term and is in class
    
            for row in train.itertuples():
                if row[6] == c:
                    if t in row[7]: N11 += 1
                    else: N01 += 1
                else:
                    if t in row[7]: N10 += 1
                    else: N00 += 1
    
            N1x = N10 + N11
            Nx1 = N01 + N11
            N0x = N00 + N01
            Nx0 = N00 + N10

            try: w = (N11/N) * math.log((N*N11)/(N1x*Nx1))
            except: w = 0
            try: x = (N01/N) * math.log((N*N01)/(N0x*Nx1))
            except: x = 0
            try: y = (N10/N) * math.log((N*N10)/(N1x*Nx0))
            except: y = 0
            try: z = (N00/N) * math.log((N*N00)/(N0x*Nx0))
            except: z = 0
                
            scores[c][t] = w + x + y + z
        
    return scores

In [105]:
%time utility = computeUtilityMI(list(trainResult[0])[:100],classes)

CPU times: user 16.5 s, sys: 161 ms, total: 16.6 s
Wall time: 16.8 s


In [107]:
utility

defaultdict(<function __main__.<lambda>>,
            {u'Buitenlandse Zaken (BUZA)': defaultdict(dict,
                         {u'1173': 0.003297393102831457,
                          u'1176': 0.00661300156347153,
                          u'1179': 0.0001748205983198478,
                          u'270': 0.0014008866414684865,
                          u'273': 0.00034991866558377636,
                          u'274': 0.0001748205983198478,
                          u'276': 0.003297393102831457,
                          u'279': 0.00034991866558377636,
                          u'aangegane': 0.00034991866558377636,
                          u'absolute': 0.001229602096368891,
                          u'adviseert': 0.0010531033515205623,
                          u'afschuift': 0.0001748205983198478,
                          u'ambtsberichten': 0.0007009507439427347,
                          u'arbeidsongevallen': 0.0001748205983198478,
                          u'archieven': 0.00329739

## Exercise 5
> For each class, show the top 10 words as in Figure 13.7 in MRS.

In [132]:
for c,scores in utility.iteritems():
    print "Class: %s" % c
    for t in sorted(scores.items(), key=lambda x: x[1], reverse=True)[:10]:
        print "  %s (score: %s)" % (t[0], t[1])
    print ''

Class: Financien (FIN)
  dnb (score: 0.00872356232802)
  criminaliteitsbeeld (score: 0.00434216431158)
  beleidsinitiatieven (score: 0.00434216431158)
  kusters (score: 0.00434216431158)
  273 (score: 0.00235568942462)
  snelle (score: 0.00222581451178)
  onderwerpen (score: 0.00172232825567)
  onttrekken (score: 0.0016574704928)
  bewust (score: 0.00146052865647)
  rechterlijke (score: 0.00137353727957)

Class: Justitie (JUS)
  rechterlijke (score: 0.010755212348)
  ambtsberichten (score: 0.00977189062443)
  machtig (score: 0.00731500455745)
  china (score: 0.00486745097508)
  onbepaalde (score: 0.00486745097508)
  ontstaat (score: 0.00331073073397)
  ontstaan (score: 0.00283058575443)
  opmerking (score: 0.00263757816431)
  demotiverend (score: 0.00242914422406)
  274 (score: 0.00242914422406)

Class: Economische Zaken (EZ)
  drenthe (score: 0.00447018322928)
  verlening (score: 0.00186300203334)
  onderwerpen (score: 0.00157891742089)
  onnodige (score: 0.000783316729539)
  voorbeho

## Exercise 6

> Evaluate your classifiers using Precision, Recall and F1.

> Give a table in which you show these values for using the top 10, top 100 terms and all terms, for all of your 10 classes. Thus do feature selection per class, and use for each class the top n best features for that class. 
Also show the microaverage(s) for all 10 classes together. 
If you like you can also present this in a figure like MRS.13.8. Then compute the F1 measure for the same number of terms as in that figure.

In [279]:
def evaluationScores(terms):
    real = [True if t in class_vocabulary[c] else False for t in terms]
    call = [True if t == c else False for t in [ApplyMultinomialNB(classes, trainResult[0], trainResult[1], trainResult[2], t) for t in terms]]
    
    TP = 0
    FN = 0
    FP = 0
    for i in range(len(real)):
        if (real[i] == True) and (real[i] == call[i]): TP += 1
        if (real[i] == True) and (real[i] != call[i]): FN += 1
        if (real[i] == False) and (real[i] != call[i]): FP += 1
    p = round((float(TP) / (TP + FP)),2)
    r = round((float(TP) / (TP + FN)),2)

    return [TP,FP,FN], [p,r,computeF1(p,r)]
    
def computeF1(p,r):
    return round(((2*p*r) / p + r),2)
    
def computeMicroaverage(terms):
    TPs = sum([item[terms][0][0] for item in evaluation.values()])
    FPs = sum([item[terms][0][1] for item in evaluation.values()])
    FNs = sum([item[terms][0][2] for item in evaluation.values()])
    p = round((float(TPs) / (TPs + FPs)),2)
    r = round((float(TPs) / (TPs + FNs)),2)
    
    return [p, r, computeF1(p,r)]

class_vocabulary = defaultdict(list)
evaluation = defaultdict(lambda: defaultdict(dict))
microaverages = {}

for row in train.itertuples():
    class_vocabulary[row[6]] += row[7] 

for c in classes:
    top10 = dict(sorted(utility[c].items(), key=lambda x: x[1], reverse=True)[:10]).keys()
    evaluation[c]['top10'] = evaluationScores(top10)

    top100 = dict(sorted(utility[c].items(), key=lambda x: x[1], reverse=True)[:100]).keys()
    evaluation[c]['top100'] = evaluationScores(top100)
    
    allTerms = trainResult[0]
    evaluation[c]['all'] = evaluationScores(allTerms)

microaverages['top10'] = computeMicroaverage('top10')
microaverages['top100'] = computeMicroaverage('top100')
microaverages['all'] = computeMicroaverage('top10')

print "Precision \t\t Recall \t\t F1"
print "T10 \t T100 \t all \t T10 \t T100 \t all \t T10 \t T100 \t all"
print "---------------------------------------------------------------------" 
for c in classes:
    print "%s \t %s \t %s \t %s \t %s \t %s \t %s \t %s \t %s \t %s" % (evaluation[c]['top10'][1][0], evaluation[c]['top100'][1][0], evaluation[c]['all'][1][0], 
                                                                        evaluation[c]['top10'][1][1], evaluation[c]['top100'][1][1], evaluation[c]['all'][1][1], 
                                                                        evaluation[c]['top10'][1][2], evaluation[c]['top100'][1][2], evaluation[c]['all'][1][2], c[:41]) 
print "---------------------------------------------------------------------"                                                                      
print "%s \t %s \t %s \t %s \t %s \t %s \t %s \t %s \t %s \t %s" % (microaverages['top10'][0], microaverages['top100'][0], microaverages['all'][0],
                                                                    microaverages['top10'][1], microaverages['top100'][1], microaverages['all'][1],
                                                                    microaverages['top10'][2], microaverages['top100'][2], microaverages['all'][2], 'Microaverages')



Precision 		 Recall 		 F1
T10 	 T100 	 all 	 T10 	 T100 	 all 	 T10 	 T100 	 all
---------------------------------------------------------------------
1.0 	 1.0 	 1.0 	 0.78 	 0.54 	 0.72 	 2.34 	 1.62 	 2.16 	 Justitie (JUS)
1.0 	 1.0 	 1.0 	 1.0 	 0.55 	 0.57 	 3.0 	 1.65 	 1.71 	 Volksgezondheid, Welzijn en Sport (VWS)
1.0 	 1.0 	 1.0 	 0.89 	 0.59 	 0.51 	 2.67 	 1.77 	 1.53 	 Buitenlandse Zaken (BUZA)
1.0 	 1.0 	 1.0 	 0.86 	 0.53 	 0.47 	 2.58 	 1.59 	 1.41 	 Verkeer en Waterstaat (VW)
1.0 	 1.0 	 1.0 	 1.0 	 0.41 	 0.37 	 3.0 	 1.23 	 1.11 	 Sociale Zaken en Werkgelegenheid (SZW)
1.0 	 1.0 	 1.0 	 1.0 	 0.29 	 0.34 	 3.0 	 0.87 	 1.02 	 Onderwijs, Cultuur en Wetenschappen (OCW)
1.0 	 1.0 	 1.0 	 1.0 	 0.32 	 0.38 	 3.0 	 0.96 	 1.14 	 Volkshuisvesting, Ruimtelijke Ordening en
1.0 	 1.0 	 1.0 	 0.8 	 0.53 	 0.32 	 2.4 	 1.59 	 0.96 	 Landbouw, Natuurbeheer en Visserij (LNV)
1.0 	 1.0 	 1.0 	 0.33 	 0.09 	 0.26 	 0.99 	 0.27 	 0.78 	 Economische Zaken (EZ)
1.0 	 1.0 	 1.0 	 0.57 	

## Exercise 7
> Reflect and report briefly about your choices in this process and about the obtained results.

#### Accountability of choices
In the assignment, we had to make several decisions. The first important decision was the normalization and selection of the ministeries. We decided to not normalize ALL the ministeries, but only the ones with the highest frequency. After some trial and error, we found the top most frequent ministeries and decided to make a selection with those. Choosing the ministeries with the highest frequency resulted in the greatest possible dataset, which would be the most interesting to work with. 

We also decided to add a column to the Pandas data frame with a precomputed tokenized string of the title, the question and the answer for each row. Saving these tokenized texts beforehand saved time in the training algorithm. 

#### Discussion of evaluation measures
The evaluation measures computed in exercise 6 are quite interesting. 
First of all, all the classes have a precision of 1. Which means there were no false positives. 
Second, there is a lot of difference between the recall (and thus the F1) scores for the top 10, top 100 and all the terms. Obviously, the scores for the top 10 terms are least precise, because of the low amount of words. If the scores increase or decrease with a higher amount of terms evaluated differs. For some classes, the scores increase, for some the scores decrease. 
The biggest challenge for this classifier is that the different classes have an overlap, especially in terminology. This in combination with the high amount of classes makes it more difficult to make correct classifications. 

#### Github Repository
[Link to our GitHub repository](https://github.com/lucaverhees/IK_zoekmachines)